## Note: Data taken from Kaggle dataset,  Ideas taken from kaggle-code, Youtube videos,and  from Neural Network and Deep Learning especialization. Course # 5 - Sequence Models - Transformers ....Dr  Andrew Ng. Table of Content.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Movie review sentiment Model.


In [ ]:
import zipfile
import os
with zipfile.ZipFile('/kaggle/input/sentiment-analysis-on-movie-reviews/train.tsv.zip','r') as zip_ref:
    zip_ref.extractall("./sentiment-analysis-on-movie-reviews/")
with zipfile.ZipFile('/kaggle/input/sentiment-analysis-on-movie-reviews/test.tsv.zip','r') as zip_ref:
    zip_ref.extractall("./sentiment-analysis-on-movie-reviews/")

 ## Reading source data
 *  Data_source sotre all the data I am going to work with.
 *  Later on I am going to split the data_source between  train and validation data
 * The content includes index, Phtase and Sentiment ( 1 to 5 )
 * The shape of data_source is   ( 156060, 2 )

In [ ]:
data_source=pd.read_table("/kaggle/working/sentiment-analysis-on-movie-reviews/train.tsv",sep='\t')
data_source=data_source[['Phrase','Sentiment']].copy()
data_source

In [ ]:
data_source.Sentiment.values

In [ ]:
data_source.index.values

In [ ]:
data_source.Phrase[:10]

In [ ]:
# split each sentence in blocks of 10 words which is going to be the size of the tokens.

dff=[len(i.split(" ")) for i in data_source.Phrase[:10]]
max(dff)

In [ ]:
dff

## Importing libraries and tools

In [ ]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFAutoModel

# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

### Splitting the data for training and validation
*  we are going to use 90% for training and 10% for validation
*  Note that X_train_data, X_validation_fdata, y_train_data and y_validation_data store data's indices

In [ ]:
# train_test_split : Split arrays or matrices into random train and test subsets
X_train_data, X_validation_data, y_train_data, y_validation_data = train_test_split(data_source.index.values,  # sequence number from 1 to 156060
                                                  data_source.Sentiment.values,   # values 1 to 5
                                                  test_size=0.15,    # should be between 0.0 and 1.0  .10 = 10% for testing and 90% for training
                                                  random_state=42,   # Controls the shuffling applied to the data before applying the split
                                                  stratify=data_source.Sentiment) # data is split in a stratified fashion, using this as the class labels



### Let's take a look at X_train and X_validation data and shape

In [ ]:
print(X_train_data)
print(X_validation_data)
print(y_train_data)

In [ ]:
data_source.shape[0]

In [ ]:
#sns.countplot(data_source)
plt.xlabel('Review Score')

### in Data_source file , assigning data-type 'training' to Training data and 'validation' to  validation data
* I  use X_train_data  and X_validation_data to localise the corresponding records within the file  data_source

In [ ]:
# in Data_source file , assigning data-type 'training' to Training data and 'validation' to  validation data

data_source['data_type'] = ['not_set']*data_source.shape[0]
data_source.loc[X_train_data, 'data_type'] = 'training'
data_source.loc[X_validation_data, 'data_type'] = 'validation'

In [ ]:
print(X_train_data.shape)
print(X_validation_data)
print(y_train_data)

In [ ]:
data_source.isnull().sum()

In [ ]:
X_train_data.shape, X_validation_data.shape


In [ ]:
data_source[data_source.data_type=='validation'].Phrase

In [ ]:
print(data_source.Sentiment.values)
print(data_source.Phrase.values)

## Defining parameters for the model
*  Note that the bert model and tokenizer function is needed tp create the dataset

In [ ]:
max_token_length = max(dff)+3   # Max length of tokens
number_of_samples = len(data_source)
bert = 'bert-base-cased'
config = BertConfig.from_pretrained(bert)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = bert, config = config)


### Defining sentiment values or labels ( 1,2,3,4,5 )

In [ ]:
data_source["Sentiment"].values

In [ ]:
data_source['Phrase'].values

In [ ]:
sent_values_array = data_source["Sentiment"].values
len(sent_values_array)
#sent_values_array.max()

#### This function receives 3 parameters and return two parameters:  One dictionary {'input_ids', 'attention_mask'},  labels
* Note that 'input_ids and  'attention_mask' will be the model's input data

In [ ]:
def map_function(input_ids, masks,labels):    # receives 3 tuples and returns 2 
    return {'input_ids': input_ids, 'attention_mask': masks},labels

## Defining train_dataset
*  Tokenization is the process of breaking up a string into tokens. Commonly, these tokens are words, numbers, and/or punctuation. The tensorflow_text package provides a number of tokenizers available for preprocessing text required by your text-based models. By performing the tokenization in the TensorFlow graph, you will not need to worry about differences between the training and inference workflows and managing preprocessing scripts.

In [ ]:
# to_categorical : Converts a class vector (integers) to binary class matrix
# converts training sentiment labels in to a binary  matrix  of shape  #items x # values    140454,  5 
x_senti = to_categorical(data_source[data_source.data_type=='training'].Sentiment)  # x_senti.shape  140454,5

# Tokenize the input 
x = tokenizer(
    text=data_source[data_source.data_type=='training'].Phrase.to_list(), # list of all Phrases 
    add_special_tokens=True,
    max_length=max_token_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

train_dataset=tf.data.Dataset.from_tensor_slices((x['input_ids'], x['attention_mask'], x_senti)) # creates tensor  of 512,84,5



In [ ]:
train_dataset = train_dataset.map(map_function)

In [ ]:
batch_size = 32
# shuffle and batch - dropping any remaining samples that don't cleanly
train_dataset = train_dataset.shuffle(1000).batch(batch_size, drop_remainder=True) #  created object of shape 32x 84  32 is the batch size

In [ ]:
x_senti.shape

In [ ]:
train_dataset

### Let's check the parameters of dataset:  x[input_ids], x[attention_mask],  x_senti

In [ ]:
train_dataset

In [ ]:
x['attention_mask']

In [ ]:
x_senti.shape

In [ ]:
x['input_ids']

In [ ]:
x

In [ ]:
len(train_dataset)

## Defining validation_dataset

In [ ]:
y_senti = to_categorical(data_source[data_source.data_type=='validation'].Sentiment)

# Tokenize the input 
v= tokenizer(
    text=data_source[data_source.data_type=='validation'].Phrase.to_list(),
    add_special_tokens=True,
    max_length=max_token_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

validation_dataset = tf.data.Dataset.from_tensor_slices((v['input_ids'], v['attention_mask'], y_senti))
validation_dataset = validation_dataset.map(map_function)
validation_dataset = validation_dataset.shuffle(1000).batch(batch_size, drop_remainder=True)

In [ ]:
max_token_length

In [ ]:
# let's create a much nicer confusion matrix
def plot_confusion_matrix(y_test, y_pred, classes=None):
  import itertools
  figsize = (20, 20)
  cm = confusion_matrix(y_test, tf.round(y_pred))

  # let's normalize confusion matrix
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
  n_classes = cm.shape[0]

  # Let's prettify it
  fig , ax = plt.subplots(figsize=figsize)

  # create matrix plot
  cax = ax.matshow(cm, cmap=plt.cm.Blues)
  fig.colorbar(cax)

  
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axis
  ax.set(title="Confusion Matrix",
          xlabel = "Predicted Label",
          ylabel = "True Label",
          xticks = np.arange(n_classes),
          yticks = np.arange(n_classes),
          xticklabels = labels,
          yticklabels = labels)

  # set X-axis labels to bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Adjust label size
  ax.yaxis.label.set_size(20)
  ax.xaxis.label.set_size(20)
  ax.title.set_size(20)


  # set threshold for different colors
  threshold = ( cm.max() + cm.min()) /2. 

  # plot text in each cell

  for i,j  in itertools.product(range(cm.shape[0]),  range(cm.shape[1])):
    plt.text(j, i, f"{cm[i,j]} ({cm_norm[i,j]*100:.1f}%)",
                               horizontalalignment = "center",
                               color = "white" if cm[i,j] > threshold else "black",
                               size = 15)

In [ ]:
# Function to Plot the validation and training data separately

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

## Defining layers that will fit the model

In [ ]:
input_ids = tf.keras.Input(shape=(max_token_length,), name='input_ids', dtype='int32') # parameters of input layer
attention_mask = tf.keras.Input(shape=(max_token_length,), name='attention_mask', dtype='int32')  # parameters of attention mask 
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}  # inputs will be the input of our model..  
bert = TFAutoModel.from_pretrained('bert-base-cased')
embeddings = bert.bert(inputs)[1]  #  access pooled activations with [1]  pool from 3 tensors to expecting  2 tensors
xis = tf.keras.layers.Dense(1024,activation='relu')(embeddings)   # intermediate dense layer
yhat = tf.keras.layers.Dense(sent_values_array.max()+1, activation='softmax', name='outputs')(xis)  # final output layer


In [ ]:
input_ids

## Building the model

In [ ]:
#model = tf.keras.Model(inputs=[train_dataset,mask], outputs=yhat)
model = tf.keras.Model(inputs= inputs, outputs=yhat)   # inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
model.summary()

### Defining Adam optimizer, loss and accuracy functions..

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=2e-5, decay=1e-6)    #  lr=1e-5
loss = tf.keras.losses.CategoricalCrossentropy()
accuracy = tf.keras.metrics.CategoricalAccuracy('accuracy')

### Compiling the model

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy])

In [ ]:
validation_dataset


## Trainig the model
*  Using Training and validation data

In [ ]:
history1 = model.fit(
    train_dataset,  #train_dataset,
    validation_data= validation_dataset,  #  validatio dataset
    epochs=8)

In [ ]:
plot_loss_curves(history1)

## Saving the model

In [ ]:
model.save('./sentiment-analysis-on-movie-reviews/Movie_sentiment_analysis_model')

In [ ]:
#model.save('Movies_sentiment_analysis_model.h5')

## Reading test data

In [ ]:
test=pd.read_table("/kaggle/working/sentiment-analysis-on-movie-reviews/test.tsv",sep='\t')

In [ ]:
test

In [ ]:
data_source[data_source.data_type=='validation'].Phrase

## Tokenizing test dataset

In [ ]:
x_test = tokenizer(
    text=test.Phrase.to_list(),
    add_special_tokens=True,
    max_length=max_token_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
x_test

In [ ]:
test_items=tf.data.Dataset.from_tensor_slices((x_test['input_ids'],x_test['attention_mask']))

In [ ]:
def map_func(input_ids, masks):
    return {'input_ids': input_ids, 'attention_mask': masks}

test_items = test_items.map(map_func)
test_items = test_items.batch(32)

In [ ]:
test_items

##  Making predictions

In [ ]:
predictions=model.predict(test_items).argmax(axis=-1)

In [ ]:

prediction2=model.predict(validation_dataset).argmax(axis=-1)


In [ ]:
prediction2.min()

In [ ]:
pred=prediction2+1
pred.max()


In [ ]:
y_validation=y_validation_data[:23392]
y_validation.min()

In [ ]:
y_validation.shape, pred.shape

In [ ]:
sentiments = [0,1,2,3,4]
#test_sent_array = test["Sentiment"].values
from sklearn.metrics import confusion_matrix

plot_confusion_matrix(y_test=y_validation,
                      y_pred=prediction2,
                      classes=sentiments)

## Conclusions and acknowledgements*

## Creating the submission file

In [ ]:
submission = pd.DataFrame()
submission['PhraseId'] = test['PhraseId']
submission['Sentiment'] = predictions
submission.to_csv("submission.csv", index=False)
submission.head()

In [ ]:
new_model =load_model('./sentiment-analysis-on-movie-reviews/Movie_sentiment_analysis_model')
new_model.summary()


In [ ]:
%load_ext tensorboard

In [ ]:
pip install -U tensorboard-plugin-profile


In [ ]:
%tensorboard --port=5036 --logdir tb_callback_dir